In [1]:
import os, platform, socket, subprocess, shutil, json, sys

def run(cmd):
    try:
        out = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, check=False)
        return out.stdout.strip()
    except Exception as e:
        return f"<error running {cmd}: {e}>"

print("==== VERIFY WE'RE ON THE SERVER ====")
print("Python executable:", sys.executable)
print("Hostname        :", socket.gethostname())
print("User            :", run(["whoami"]))
print()

print("==== OS / KERNEL ====")
print(run(["uname", "-a"]))
print()

print("==== CPU ====")
if shutil.which("lscpu"):
    print(run(["lscpu"]))
else:
    # Minimal fallback
    print("Machine:", platform.machine())
    print("Processor:", platform.processor())
    print("Logical CPUs:", os.cpu_count())
print()

print("==== MEMORY ====")
if shutil.which("free"):
    print(run(["free", "-h"]))
else:
    # POSIX fallback (approx)
    try:
        pages = os.sysconf("SC_PHYS_PAGES")
        psize = os.sysconf("SC_PAGE_SIZE")
        print(f"Total RAM: {pages * psize / 1e9:.2f} GB")
    except Exception:
        print("free not available and sysconf failed.")
print()

print("==== DISKS ====")
if shutil.which("lsblk"):
    print(run(["lsblk", "-o", "NAME,SIZE,TYPE,MOUNTPOINT,FSTYPE"]))
else:
    print(run(["df", "-h"]))
print()

print("==== GPU (NVIDIA) ====")
if shutil.which("nvidia-smi"):
    print(run(["nvidia-smi", "--query-gpu=index,name,driver_version,memory.total,memory.used", "--format=csv,noheader"]))
else:
    # Try to at least show PCI devices mentioning NVIDIA/AMD
    if shutil.which("lspci"):
        lines = run(["lspci"]).splitlines()
        gpus = [l for l in lines if any(v in l.lower() for v in ["nvidia", "amd", "advanced micro devices", "radeon"])]
        print("\n".join(gpus) if gpus else "No nvidia-smi; lspci found no obvious GPU entries.")
    else:
        print("No nvidia-smi and lspci not available.")
print()

print("==== NETWORK ====")
if shutil.which("ip"):
    print(run(["ip", "-brief", "addr"]))
else:
    print(run(["ifconfig"]) or "Neither ip nor ifconfig available.")

==== VERIFY WE'RE ON THE SERVER ====
Python executable: /home/s0203301/jupyter_server/3_8_jupyter/bin/python
Hostname        : idistributedartificialintelligence-Group2
User            : s0203301

==== OS / KERNEL ====
Linux idistributedartificialintelligence-Group2 6.8.0-83-generic #83~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Sep  9 18:19:47 UTC 2 x86_64 x86_64 x86_64 GNU/Linux

==== CPU ====
Architecture:                         x86_64
CPU op-mode(s):                       32-bit, 64-bit
Address sizes:                        40 bits physical, 48 bits virtual
Byte Order:                           Little Endian
CPU(s):                               20
On-line CPU(s) list:                  0-19
Vendor ID:                            AuthenticAMD
Model name:                           QEMU Virtual CPU version 2.5+
CPU family:                           15
Model:                                107
Thread(s) per core:                   1
Core(s) per socket:                   10
Socket(s):      

# Carla API Voorbeeld

In [ ]:
import carla

client = carla.Client('localhost', 2000)
client.set_timeout(10.0) # seconds
world = client.get_world()